# Train more sophisticated model with national data, which is begger.
2024/06/27 comment: 思ったよりデータが小さい。25歳までは一歳ずつなのにそれ以上は５歳刻みで使いにくい。とりあえず放置。以前のモデルのままでいく。

In [5]:
#try to create a simple lightGBM model with physical features: height, weight, age, sex.
#firstly, import the necessary libraries
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier, log_evaluation
import lightgbm as lgbm
import matplotlib.pyplot as plt
#import seaborn as sns

In [112]:
#load the data and EDA a bit to see what we have
df = pd.read_csv('japanesePhysicaldata.csv')
df.head(10)



,Unnamed: 0,Unnamed: 1,/身長体重,身長(cm),身長(cm).1,身長(cm).2,体重(kg),体重(kg).1,体重(kg).2,身長(cm).3,身長(cm).4,身長(cm).5,体重(kg).3,体重(kg).4,体重(kg).5
0,時間軸(年次),年齢階級,/表章項目,人数【人】,平均値,標準偏差,人数【人】,平均値,標準偏差,人数【人】,平均値,標準偏差,人数【人】,平均値,標準偏差
1,2019年,総数,NaN,"2,342",163.1,16.5,"2,332",62.7,17.2,"2,697",151.3,13.6,"2,681",50.8,12.5
2,2019年,1歳,NaN,15,79.6,4.7,15,10.3,1.4,14,76.6,4.2,17,9.7,1.4
3,2019年,2歳,NaN,15,89,4.2,15,12.2,1.5,23,88.2,3.5,23,12.3,1.5
4,2019年,3歳,NaN,8,95.6,4.1,8,13.8,0.9,8,95.7,3.9,8,13.9,1.1
5,2019年,4歳,NaN,13,103.7,4.8,13,16.4,1.9,20,102.9,5.3,20,16.5,2.6
6,2019年,5歳,NaN,19,110.5,5.3,19,18.2,3.1,25,107.5,4.8,25,17.7,2.6
7,2019年,6歳,NaN,15,114.9,4.3,15,20.6,3.3,18,114.7,4.8,17,20.4,2.6
8,2019年,7歳,NaN,28,122.7,4.3,28,24.7,5.8,21,121.1,5,22,21.8,2.8
9,2019年,8歳,NaN,24,126.3,4.9,24,25.8,6.3,21,125.5,6.1,22,25.9,4.6


In [113]:
df.rename(columns={'Unnamed: 0': 'year'}, inplace=True)
df.rename(columns={'Unnamed: 1': 'age'}, inplace=True)

In [121]:
df['age'] = df['age'].str.replace('歳', '')
df.tail(10)

,year,age,/身長体重,身長(cm),身長(cm).1,身長(cm).2,体重(kg),体重(kg).1,体重(kg).2,身長(cm).3,身長(cm).4,身長(cm).5,体重(kg).3,体重(kg).4,体重(kg).5
381,2010年,50-59,NaN,453,168,6.1,451,68.2,10.4,567,155.2,5.4,564,54.1,8.6
382,2010年,60-69,NaN,650,165.3,6.1,649,64.6,9.1,765,151.8,5.5,763,53.4,8.3
383,2010年,70以上,NaN,638,160.9,6.3,637,59.9,9.4,833,147.7,6.2,836,50.4,8.8
384,2010年,（再掲）20以上,NaN,"2,743",166.8,7.2,"2,732",65.7,11.1,"3,399",153.6,6.9,"3,367",52.9,9
385,2010年,（再掲）20-29,NaN,218,170.6,6,215,65.1,12,259,158.1,5.4,252,51,9
386,2010年,（再掲）60-64,NaN,335,166.4,6.2,334,65.7,9.2,377,152.7,5.5,376,54.2,8.7
387,2010年,（再掲）65-69,NaN,315,164.1,5.8,315,63.4,8.7,388,150.9,5.3,387,52.6,7.9
388,2010年,（再掲）70-74,NaN,261,162.8,6.2,260,62.3,9.5,316,149.9,5.4,316,52.2,8.7
389,2010年,（再掲）75-79,NaN,190,160.2,5.8,190,59.5,9.2,251,147.1,6,251,50.5,8.5
390,2010年,（再掲）80以上,NaN,187,159.1,6.3,187,56.9,8.4,266,145.6,6.5,269,48.1,8.8


In [114]:
df.columns

Index(['year', 'age', '/身長体重', '身長(cm)', '身長(cm).1', '身長(cm).2', '体重(kg)',
       '体重(kg).1', '体重(kg).2', '身長(cm).3', '身長(cm).4', '身長(cm).5', '体重(kg).3',
       '体重(kg).4', '体重(kg).5'],
      dtype='object')

In [115]:
df = df[df['Unnamed: 0'] == '2019年']
df = df.drop([x for x in range(33,40)])
df.tail(10)

KeyError: 'Unnamed: 0'

In [116]:
index_forthisdata = df.iloc[:, :2]

index_forthisdata.head(10)

,year,age
0,時間軸(年次),年齢階級
1,2019年,総数
2,2019年,1歳
3,2019年,2歳
4,2019年,3歳
5,2019年,4歳
6,2019年,5歳
7,2019年,6歳
8,2019年,7歳
9,2019年,8歳


In [117]:
df_men = df.iloc[:, 3:9]
df_women = df.iloc[:, 9:]
print(f'men data {df_men.head()})')
print(f'wemen data: {df_women.tail()}')

men data   身長(cm) 身長(cm).1 身長(cm).2 体重(kg) 体重(kg).1 体重(kg).2
0  人数【人】      平均値     標準偏差  人数【人】      平均値     標準偏差
1  2,342    163.1     16.5  2,332     62.7     17.2
2     15     79.6      4.7     15     10.3      1.4
3     15       89      4.2     15     12.2      1.5
4      8     95.6      4.1      8     13.8      0.9)
wemen data:     身長(cm).3 身長(cm).4 身長(cm).5 体重(kg).3 体重(kg).4 体重(kg).5
386      377    152.7      5.5      376     54.2      8.7
387      388    150.9      5.3      387     52.6      7.9
388      316    149.9      5.4      316     52.2      8.7
389      251    147.1        6      251     50.5      8.5
390      266    145.6      6.5      269     48.1      8.8


In [118]:
df_men = index_forthisdata.join(df_men, lsuffix='index')
df_men.head()

,year,age,身長(cm),身長(cm).1,身長(cm).2,体重(kg),体重(kg).1,体重(kg).2
0,時間軸(年次),年齢階級,人数【人】,平均値,標準偏差,人数【人】,平均値,標準偏差
1,2019年,総数,"2,342",163.1,16.5,"2,332",62.7,17.2
2,2019年,1歳,15,79.6,4.7,15,10.3,1.4
3,2019年,2歳,15,89,4.2,15,12.2,1.5
4,2019年,3歳,8,95.6,4.1,8,13.8,0.9


In [110]:
df_women = index_forthisdata.join(df_women, lsuffix='index')
df_women.tail(20)

,Unnamed: 0,Unnamed: 1,身長(cm).3,身長(cm).4,身長(cm).5,体重(kg).3,体重(kg).4,体重(kg).5
13,2019年,12歳,19,150.9,5.7,19,41.9,5.6
14,2019年,13歳,28,154.8,6,27,48.8,7.6
15,2019年,14歳,20,155.5,5.6,21,48.4,6.1
16,2019年,15歳,21,159.2,5.9,21,51.2,6.9
17,2019年,16歳,16,158,6.3,16,48.9,4.8
18,2019年,17歳,17,158.4,4.5,16,52.6,7.5
19,2019年,18歳,13,156,6.4,13,49.6,4.8
20,2019年,19歳,15,156.7,7.4,15,48.7,7.5
21,2019年,20歳,14,158.6,4.2,14,49,5.3
22,2019年,21歳,10,158.7,5.6,10,54.6,9


# we have done data preprocessing, got data: Year 2019, height, age, sex, weight data.Congrats!!

In [ ]:
#main function
#read, reshape, drop some and split the data

def preprocessor_book(file_path):
    df = pd.read_csv(file_path)
    #drop the columns that are not needed


    #reshape or normalize or change the data type if needed
    df_feature = ??

    return df_feature #return the feature data frame

In [ ]:
df_train = preprocessor_book('**.csv')
df_test = preprocessor_book('**.csv')

Conduct cross validation.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import log_loss


# conduct the cross validation, 5fold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for idx_1, idx_2 in kf.split(df_train):
    train_data = df_train.iloc[idx_1]
    test_data = df_train.iloc[idx_2]
    X_train = train_data.drop(columns=['target'])
    y_train = train_data['target']
    X_test = test_data.drop(columns=['target'])
    y_test = test_data['target']
    model = LGBMClassifier(n_estimators=1000, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2, reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='logloss', early_stopping_rounds=100, verbose=True)
    y_pred = model.predict(X_test)
    print(log_loss(y_test, y_pred))
    #print(model.feature_importances_)
    #plt.barh(X_train.columns, model.feature_importances_)
    #plt.show()
    #print(model.feature_importances_)



'scores = -1 * cross_val_score(my_pipeline, OH_X_train, y,\n                              cv=20,\n                              scoring=\'neg_mean_absolute_error\')\n\nprint("MAE scores:\n", scores)\n'

# Prepare the data

In [ ]:
X = df_train.drop(['row_id','target'],axis=1)
y = df_train['target']

# Train with .fit()